In [13]:
from auto_everything.io import IO
io_ = IO()

proto_string = io_.read("./test_protobuff_code.proto")

In [14]:
import re

In [23]:
from pprint import pprint


code_block_list = re.findall(r"(?P<type>\w+)\s+(?P<object_name>\w+)\s+\{(?P<properties>(\s*.*?\s*)+)\}", proto_string, re.DOTALL)
code_block_list = [
            [string for string in one][:3]
            for one in code_block_list
        ]

pprint(code_block_list[0])
print()
pprint(code_block_list[1])
print()
pprint(code_block_list[2])

['service',
 'Greeter',
 'rpc SayHello (hello_request) returns (HelloReply);\n'
 '    rpc SendVoice(stream Voice_Request) returns (Empty);\n'
 '    rpc GetCurrentUsersUUID(Empty) returns (CurrentUsersUUIDReply);\n'
 '    rpc say_hello (hello_request) returns (HelloReply);']

['enum', 'UserStatus', 'OFFLINE = 0;\n    ONLINE = 1;']

['message', 'Empty', '']


In [34]:
from typing import Any


new_parsed_object_list: list[Any] = []
for one in code_block_list.copy():
    if len(one) == 3:
        type_ = one[0]
        class_name = one[1]
        content = one[2]

        if (type_ == "service"):
            new_parsed_object_list.append([type_.strip(), class_name.strip(), content])
        else:
            property_text = " " + content.strip()
            if len(property_text) == 0:
                continue

            property_list = []
            if (type_ == "message"):
                property_list = re.findall(r"(?P<feature>\w*)\s+(?P<type>\w+)\s+(?P<property>\w+)\s+=\s+\d+;", property_text)
                #print(property_list)
            elif (type_ == "enum"):
                property_list = re.findall(r"(?P<property>\w+)\s+=\s+\d+;", property_text)
                property_list = [('','string', one) for one in property_list]
                #print(property_list)

            new_parsed_object_list.append([type_.strip(), class_name.strip(), property_list])

        #print(new_parsed_object_list[-1])

pprint(new_parsed_object_list)

[['service',
  'Greeter',
  'rpc SayHello (hello_request) returns (HelloReply);\n'
  '    rpc SendVoice(stream Voice_Request) returns (Empty);\n'
  '    rpc GetCurrentUsersUUID(Empty) returns (CurrentUsersUUIDReply);\n'
  '    rpc say_hello (hello_request) returns (HelloReply);'],
 ['enum', 'UserStatus', [('', 'string', 'OFFLINE'), ('', 'string', 'ONLINE')]],
 ['message', 'Empty', []],
 ['message',
  'hello_request',
  [('', 'string', 'name'), ('', 'UserStatus', 'user_status')]],
 ['message', 'HelloReply', [('', 'string', 'message')]],
 ['message',
  'Voice_Request',
  [('', 'string', 'uuid'),
   ('', 'int64', 'timestamp'),
   ('', 'string', 'voice')]],
 ['message', 'VoiceReply', [('', 'string', 'error'), ('', 'bool', 'success')]],
 ['message', 'CurrentUsersUUIDReply', [('repeated', 'string', 'uuid')]]]


In [42]:
class_name_list = [one[1].strip() for one in new_parsed_object_list]
for one in class_name_list:
    if class_name_list.count(one) > 1:
        raise Exception("You must make sure there has no duplicated class/message name.")

In [45]:
import json

defination_tree = {}
for one in new_parsed_object_list.copy():
    code_block_type = one[0].strip()
    class_name = one[1].strip()
    variable_list = one[2]

    if code_block_type == "service":
        continue

    name_list = [one[2] for one in variable_list]
    for one in name_list:
        if name_list.count(one) > 1:
            raise Exception("You must make sure there has no duplicated variable name.")

    defination_tree[class_name] = {}
    for one_variable in variable_list:
        feature = one_variable[0]
        type = one_variable[1]
        name = one_variable[2]

        # code_block_type_for_this_variable = code_block_type
        # for one in new_parsed_object_list.copy():
        #     temp_code_block_type = one[0].strip()
        #     temp_class_name = one[1].strip()
        #     if temp_code_block_type != "enum":
        #         continue
        #     if type == temp_class_name:
        #         code_block_type_for_this_variable = temp_code_block_type
        #         break

        defination_tree[class_name][name] = {
            # "code_block_type": code_block_type_for_this_variable,
            "type": type, 
            "is_list": True if feature == "repeated" else False,
            "name": name,
            "feature": feature
        }

print(json.dumps(defination_tree, indent=4, sort_keys=False))

{
    "UserStatus": {
        "OFFLINE": {
            "type": "string",
            "is_list": false,
            "name": "OFFLINE",
            "feature": ""
        },
        "ONLINE": {
            "type": "string",
            "is_list": false,
            "name": "ONLINE",
            "feature": ""
        }
    },
    "Empty": {},
    "hello_request": {
        "name": {
            "type": "string",
            "is_list": false,
            "name": "name",
            "feature": ""
        },
        "user_status": {
            "type": "UserStatus",
            "is_list": false,
            "name": "user_status",
            "feature": ""
        }
    },
    "HelloReply": {
        "message": {
            "type": "string",
            "is_list": false,
            "name": "message",
            "feature": ""
        }
    },
    "Voice_Request": {
        "uuid": {
            "type": "string",
            "is_list": false,
            "name": "uuid",
            "feature":

In [29]:
import sys
sys.path.insert(0, './build/test_protobuff_code.py')

In [31]:
import test_protobuff_code 
hello = test_protobuff_code.HelloReply()

In [33]:
hello.from_dict({
    'message': "hi"
})

In [34]:
hello

HelloReply(message='hi')